# Construir un sistema RAG para QA sobre un dominio especifico.


**Dataset:** [`wiki_qa`](https://huggingface.co/datasets/wiki_qa) (los subsets entregados están preprocesados para que sea más facil trabajarlos, no son exactamente iguales a los originales del dataset).

**Objetivo:** Implementar y evaluar un sistema *Retrieval-Augmented Generation* (RAG) que responda preguntas usando un corpus de oraciones de Wikipedia. Debe comparar la calidad de las respuestas del sistema RAG vs un baseline sin retrieval, tanto frente a preguntas del dominio de la base de conocimiento vectorial como fuera de ella.

## 1) Carga de Datos

Los sets de train y test son mezclas entre los subsets originales del dataset WikiQA. Notar que cada pregunta tiene su propio `question_id`, sin embargo una misma pregunta puede tener varias respuestas en el dataset.

### 1.1 Carga de los datos

In [1]:
import pandas as pd
### Carga de los datos:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [2]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232 entries, 0 to 1231
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     1232 non-null   object
 1   question        1232 non-null   object
 2   document_title  1232 non-null   object
 3   answer          1232 non-null   object
dtypes: object(4)
memory usage: 38.6+ KB


In [3]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     241 non-null    object
 1   question        241 non-null    object
 2   document_title  241 non-null    object
 3   answer          241 non-null    object
dtypes: object(4)
memory usage: 7.7+ KB


In [4]:
### Exploración de los datos de train:
df_train.groupby('question').size()

question
HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US    1
HOW MANY STRIPES ARE ON THE AMERICAN FLAG          2
HOW MUCH IS CENTAVOS IN MEXICO                     1
How Do You Get Hepatitis C                         1
How Works Diaphragm Pump                           1
                                                  ..
who wrote the song hallelujah                      1
who wrote the song in the mood                     1
who wrote west side story                          1
who wrote what's my name rihanna                   1
who wrote white christmas                          1
Length: 1042, dtype: int64

In [5]:
question_series = df_train['question'].str.lower()
question_series

0                           how are glacier caves formed?
1              how much are the harry potter movies worth
2                               how a rocket engine works
3       how are cholera and typhus transmitted and pre...
4                                  how did anne frank die
                              ...                        
1227               what is the main component of vaccines
1228                            what is preciosa crystal?
1229                    who are all of the jonas brothers
1230                       who is mary matalin married to
1231                            where is the brisket from
Name: question, Length: 1232, dtype: object

### 1.2 Se crean `List()` con las preguntas de train y test

In [6]:
def preguntas_unicas(df):
    """
    Función que crea una lista con las preguntas únicas de un dataframe.
        Input: dataframe con columna 'question'
        Output: list con preguntas únicas
    """
    list_preguntas = list()
    for pregunta in df['question'].unique():
        list_preguntas.append(pregunta)
    return list_preguntas

questions_train = preguntas_unicas(df_train)
questions_test = preguntas_unicas(df_test)

In [7]:
### Tamaño de los sets de preguntas únicas:

print("Número de preguntas únicas en set de train:", len(questions_train))
print("Número de preguntas únicas en set de test:", len(questions_test))

Número de preguntas únicas en set de train: 1042
Número de preguntas únicas en set de test: 200


In [8]:
questions_train[21]

'What did the augurs use to interpret the will of the gods?'

### 1.3 Creación de base de conocimiento set train

In [9]:
df_train.query('question == @questions_train[20]')['question']

22    how do forensic auditors examine financial rep...
23    how do forensic auditors examine financial rep...
24    how do forensic auditors examine financial rep...
Name: question, dtype: object

In [10]:
df_train.query('question == @questions_train[40]')

,question_id,question,document_title,answer
53,Q188,what area code is 479,Area code 479,Area code 479 is the telephone area code servi...
54,Q188,what area code is 479,Area code 479,"Area code 479 serves Benton , Carroll (split w..."


Observación:
---

Notar que dentro del set de datos existen preguntas repetidas, por tanto, existen 2 o más respuestas. Ahora bien, para que la lista `Facts` tenga la misma tamaño que la lista de palabras únicas se eligirá aleatoriamente una respuesta entre las posibles. A continuación se deja la implementación.


In [11]:
def list_facts(df, list_questions):
    """
    Función que crea una lista con una respuesta aleatoria por cada pregunta única.
        Input: dataframe con columnas 'question' y 'answer', 
               lista con preguntas únicas
        Output: list con respuestas (facts)
    """
    import random
    list_facts = list()
    for pregunta in list_questions:
        posibles_respuestas = df.query('question == @pregunta')['answer'].tolist()
        ## Pregunta + respuesta aleatoria:
        incluir = f'{pregunta} : {random.choice(posibles_respuestas)}'
        ### Incorporar a la lista:
        list_facts.append(incluir)
    return list_facts

Facts = list_facts(df_train, questions_train)


## 2) Indexación y *retrieval*

Se construye un indice de embeddings usando las librerías sentence-transformers y faiss.

**Pasos**:
1. generar una base de conocimiento vectorial calculado los embeddings del corpus de respuestas (a.k.a lista `facts`)
2. implemente la función `retrieve(query, k)` que recupera los k extractos más relevantes (es decir, similares) a la query.

In [12]:
### Modelo de embeddings:
from sentence_transformers import SentenceTransformer, util
import faiss

embedder = SentenceTransformer('all-MiniLM-L6-v2')

corpus_embeddings = embedder.encode(Facts,
                                    normalize_embeddings=True,
                                    convert_to_tensor=True)


In [ ]:
def retrieve(query, k):
    """
    Función que obtiene las k respuestas más similares a la pregunta dada, para estos fines utiliza faiss para la búsqueda eficiente.
        Input: query (str), k (int)
        Output: list con las k respuestas más similares
    """
    ## Paso 1 : Embedding de la pregunta (corresponde al mismo modelo de embeddings que el corpus)
    query_embedding = embedder.encode(query,
                                      normalize_embeddings=True,
                                      convert_to_tensor=True)
    ## Paso 2 : Búsqueda de las k respuestas más similares, mediante faiss

    index = faiss.IndexFlatIP(corpus_embeddings.shape[1])

    index.add(corpus_embeddings.cpu().detach().numpy())

    D, I = index.search(query_embedding.cpu().detach().numpy().reshape(1, -1), k)
    
    ### Paso 3 : Obtener las k respuestas más similares
    hits = []
    for i in range(k):
        hits.append(Facts[I[0][i]])
    return hits

### 2.1 Testeo del  `Retriver`

In [14]:
### Ejemplo de uso:
retrieve('how do forensic auditors examine financial reporting', 4)

['how do forensic auditors examine financial reporting : The purpose of an audit is provide and objective independent examination of the financial statements, which increases the value and credibility of the financial statements produced by management, thus increase user confidence in the financial statement, reduce investor risk and consequently reduce the cost of capital of the preparer of the financial statements.',
 'who can file suspicious activity report : In United States financial regulation , a suspicious activity report (or SAR) is a report made by a financial institution to the Financial Crimes Enforcement Network (FinCEN), an agency of the United States Department of the Treasury , regarding suspicious or potentially suspicious activity.',
 'what is a vetting process : Vetting is the process of performing a background check on someone before offering them employment, conferring an award, etc.',
 'when did proof die : In 2006, Proof was shot and killed during an altercation 


## 4) LLM 
**Instrucciones**:

Usando la librería DSPy, se definen dos módulos

1. `RAG`: Módulo que recibe una query junto a k extractos relevantes como contexto, para generar una respuesta a la query.

2. `zero_shot`: Módulo que recibe una query y genera una respuesta a la misma, sin contar con documentos de contexto.


In [15]:
import dspy
import openai
import pass_openai_key
from typing import List, Optional, Union


### Definición del modelo de lenguaje:

lm = dspy.LM("openai/gpt-4o-mini", api_key = pass_openai_key.api_key, temperature=1)
dspy.configure(lm=lm)

### Paso 1 : definir las signatures:

class AWC(dspy.Signature):
    """
    Responde la pregunta apoyándote en el contexto dado.
    En caso de no encontrar evidencia en el contexto, responde de manera honesta que no sabes.
    Ejemplo:
      pregunta : "¿Cuál es la capital de Francia?"
      contexto : "Francia es un país en Europa. Su capital es París."
      respuesta: "París"
    """
    question = dspy.InputField(desc="Pregunta del usuario", prefix="Pregunta:")
    context  = dspy.InputField(desc="Contexto (evidencia)",  prefix="Contexto:")
    answer   = dspy.OutputField(desc="Respuesta breve", prefix="Respuesta:")

class zero_shot_qa(dspy.Signature):
    """
    Responde la pregunta sin contexto.

    ejemplo de uso:
    pregunta : "¿Cuál es la capital de Francia?"
    respuesta : "París" --->(debería ser capaz de responder sin contexto)
    """
    question = dspy.InputField(desc="Pregunta del usuario", prefix="Pregunta:")
    answer   = dspy.OutputField(desc="Respuesta breve", prefix="Respuesta:")

### Paso 2 : definir los módulos:

class RAG(dspy.Module):
    """
    Si contexts está vacío o solo trae cadenas vacías y fallback=True,
    responde en modo Zero-shot. Si hay contexto, usa AWC (anclado al contexto).
    """
    def __init__(self, fallback: bool = True):
        super().__init__()
        self.fallback = fallback
        self.rag_pred = dspy.Predict(AWC)
        self.zs_pred  = dspy.Predict(zero_shot_qa)

    def forward(self, question: str, contexts=None, k: int = 4):
        contexts = contexts or []
        # Normaliza a texto y corta a k
        ctxs = [
            (c.get("text") if isinstance(c, dict) and isinstance(c.get("text"), str) else str(c))
            for c in contexts[:k]
        ]
        # Revisa si hay contexto no vacío
        has_context = any(s.strip() for s in ctxs)

        ### Si no hay contexto y fallback está activado:
        if not has_context and self.fallback:
            out = self.zs_pred(question=question)
            return dspy.Prediction(answer=out.answer)

        context_text = "\n\n".join(f"[{i+1}] {c}" for i, c in enumerate(ctxs)) or "[Sin contexto disponible]"
        out = self.rag_pred(question=question, context=context_text)
        return dspy.Prediction(answer=out.answer)

class ZeroShot(dspy.Module):
    """
    Zero-shot module: responde preguntas sin contexto.
    """
    def __init__(self):
        super().__init__()
        self.pred = dspy.Predict(zero_shot_qa)

    def forward(self, question: str):
        out = self.pred(question=question)
        return dspy.Prediction(answer=out.answer)

### Paso 3 : Instanciar los módulos:

rag = RAG()
zero_shot = ZeroShot()

### 4.1 Testo RAG & Zero_shot (Con una pregunta del set)

In [16]:
test_question_1 = questions_test[10]
print("Test question:", test_question_1)
zero_shot.forward(test_question_1)

2025/10/01 00:10:51 WARNING dspy.primitives.module: Calling module.forward(...) on ZeroShot directly is discouraged. Please use module(...) instead.


Test question: what did isaac newton do


Prediction(
    answer='Isaac Newton formulated the laws of motion and universal gravitation, and made significant contributions to mathematics, optics, and astronomy.'
)

In [17]:
### RAG test:
print("Test question:", test_question_1)
rag.forward(test_question_1, retrieve(test_question_1, 5))

2025/10/01 00:10:51 WARNING dspy.primitives.module: Calling module.forward(...) on RAG directly is discouraged. Please use module(...) instead.


Test question: what did isaac newton do


Prediction(
    answer='Isaac Newton was an English physicist and mathematician, widely regarded as one of the most influential scientists of all time and a key figure in the scientific revolution.'
)

### 4.2 Testo RAG & Zero_shot (Con una pregunta relacionada al dataset (no está incluida))

In [18]:
### Testeo de los módulos usando 1 pregunta de testeo, relacionada con la pregunta 10 del set de datos.
test_question_2 = 'what are financial reports'
print("Test question:", test_question_2)

rag.forward(test_question_2, retrieve(test_question_2, 3))


2025/10/01 00:10:52 WARNING dspy.primitives.module: Calling module.forward(...) on RAG directly is discouraged. Please use module(...) instead.


Test question: what are financial reports


Prediction(
    answer='Financial reports are documents that present the financial status of an organization, including financial statements that provide objective examinations of financial performance, aimed at increasing credibility and user confidence.'
)

In [19]:
zero_shot.forward(test_question_2)

2025/10/01 00:10:54 WARNING dspy.primitives.module: Calling module.forward(...) on ZeroShot directly is discouraged. Please use module(...) instead.


Prediction(
    answer='Financial reports are formal records that outline the financial activities and position of a business, organization, or individual, typically including balance sheets, income statements, and cash flow statements.'
)

### 4.3 Testo RAG & Zero_shot (Con una pregunta de actualidad)

In [20]:
actualidad = "¿Qué equipo tiene actualmente más puntos en la premier league?"
print("Test question:", actualidad)
rag.forward(actualidad, retrieve(actualidad, 3))

2025/10/01 00:10:54 WARNING dspy.primitives.module: Calling module.forward(...) on RAG directly is discouraged. Please use module(...) instead.


Test question: ¿Qué equipo tiene actualmente más puntos en la premier league?


Prediction(
    answer='No sé.'
)

In [21]:
print("Test question:", actualidad)
zero_shot.forward(actualidad)

2025/10/01 00:10:55 WARNING dspy.primitives.module: Calling module.forward(...) on ZeroShot directly is discouraged. Please use module(...) instead.


Test question: ¿Qué equipo tiene actualmente más puntos en la premier league?


Prediction(
    answer='No tengo acceso a información en tiempo real. Te recomiendo verificar una fuente actualizada para conocer qué equipo tiene más puntos en la Premier League actualmente.'
)


## 5) Integración y test

**Pasos:**
- Definir una función que integre el retriever y el LLM ya definidos. La función debe tomar como input una query y generar una respuesta usando RAG.
- Probar con 10 preguntas de la lista `questions_train` y 10 preguntas de la lista `questions_test` el pipeline RAG vs el modelo de lenguaje zero-shot (como baseline).

In [22]:
### Selección de preguntas random del set 10 train y 10 testeo 
import random
random.seed(123)
preguntas_random_train = random.sample(questions_train, 10)
preguntas_random_test = random.sample(questions_test, 10)

testing_questions = preguntas_random_train + preguntas_random_test

In [23]:
testing_questions

['how was the phone invented',
 'what is the role of heredity',
 'what does a laboratory in a gynecologist office consist of',
 'what is three phase electrical',
 'what is extreme right wing',
 'what day is the federal holiday for Martin Luther King Jr.',
 'what county is St. Elizabeth MO in',
 'what are the most known sports in america',
 'WHERE WAS JOHN WAYNE BORN',
 'what is level of agreement mean',
 'what happened to stevie ray vaughan',
 'how many countries are member of the eu?',
 'how many episodes of Lost were there',
 'where does ground pepper come from',
 'how many people were killed in the oklahoma city bombing',
 'what are four thirds cameras',
 'who did mr bojangles',
 'who built the globe',
 'how many pawns in chess',
 'how much is 1 tablespoon of water']

In [24]:
def pipeline_rag(query, k: int = 4):
    queries = query if isinstance(query, (list, tuple, set)) else [query]
    rows = []
    for q in queries:
        try:
            contexts = retrieve(q, k)
        except Exception as e:
            contexts, ans = [], f"[retrieve error: {e}]"
        else:
            try:
                ans = rag(question=q, contexts=contexts, k=k).answer
            except Exception as e:
                ans = f"[RAG error: {e}]"
        rows.append({"question": q, "answer_rag": ans, "contexts_used": len(contexts)})
    return pd.DataFrame(rows)

In [25]:
def pipeline_zero_shot(query):
    queries = query if isinstance(query, (list, tuple, set)) else [query]
    rows = []
    for q in queries:
        try:
            ans = zero_shot(question=q).answer
        except Exception as e:
            ans = f"[Zero-shot error: {e}]"
        rows.append({"question": q, "answer_zero_shot": ans})
    return pd.DataFrame(rows)

In [26]:
rag_df = pipeline_rag(testing_questions, k=4)

In [27]:
zero_shot_df = pipeline_zero_shot(testing_questions)

In [28]:
df_test_results = pd.merge(rag_df, zero_shot_df, on='question', how='outer')
df_test_results[['question', 'answer_rag', 'answer_zero_shot', 'contexts_used']]

,question,answer_rag,answer_zero_shot,contexts_used
0,WHERE WAS JOHN WAYNE BORN,"John Wayne was born in Winterset, Iowa.","Marion, Indiana",4
1,how many countries are member of the eu?,I don't know.,27,4
2,how many episodes of Lost were there,I don't know.,121,4
3,how many pawns in chess,I don't know.,16,4
4,how many people were killed in the oklahoma ci...,I don't know.,168,4
5,how much is 1 tablespoon of water,I don't know.,15 milliliters,4
6,how was the phone invented,The telephone was invented through the collabo...,The telephone was invented by Alexander Graham...,4
7,what are four thirds cameras,I don't know.,Four thirds cameras refer to digital cameras t...,4
8,what are the most known sports in america,The most known sports in America include baske...,"American football, basketball, baseball, socce...",4
9,what county is St. Elizabeth MO in,"St. Elizabeth is in Miller County, Missouri.","St. Elizabeth, MO is in Miller County.",4


# Conclusiones 

* En base a la experimentación, se observa que el RAG es más entrega respuesta más concretas en la medida que existe un contexto asociado. De hecho, se observa que el modelo zero_shot entrega el lugar de nacimiento de manera incorrecta (alucinación). Por lo tanto, en contextos especificos es ideal la utilización de RAG.

* Así mismo, si la respuesta la sabe la LLM debido que estuvo en su set de entrenamiento, RAG aporta información adicional, un ejemplo de esto son los demortes más comunes en EEUU, nombrando adicionalmente las ligas asociadas estos deportes. Relacionado con lo anterior RAG, reporta información más precisa, por ejemplo, cuando la pregunta hace referencia a Stevie ray vaughan, precisa que tuvo un accidente en un helicoptero, no en una "aeronave". Esto también se aborda cuando se realiza una pregunta de testeo en el apartado 4.2, preguntando '¿Qué es un reporte financiero?', observándose el mismo efecto.

* Por otra parte, en la experimentación se definió una firma para identificar aquellas respuestas que el retriever no pudo relacionar, respondiendo "no sé". Si bien, en este contexto de preguntas no sofisticadas, el modelo de lenguaje con Zero_Shot contesta, puede provocar que la IA conteste de manera equivocada cuando el dóminio a consultar es muy especifico. Vemos que en el caso del RAG la precisión es mayor.